In [ ]:
%matplotlib inline

import numpy as np

import matplotlib
import matplotlib.pyplot as plt

import os

import ipywidgets
from ipywidgets import interact

import pywicta
from pywicta.io import geometry_converter
from pywicta.io.images import image_generator
from pywicta.io.images import plot_ctapipe_image
from pywicta.io.images import plot_hillas_parameters_on_axes
from pywicta.io.images import get_mars_like_default_integrator_config
from pywicta.image.hillas_parameters import get_hillas_parameters
from pywicta.denoising.tailcut import Tailcut
from pywicta.denoising.rejection_criteria import CTAMarsCriteria

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;
// see https://github.com/ipython/ipython/issues/2172

## Make the dataset

In [ ]:
#CAM_ID = "ASTRICam"
#CAM_ID = "CHEC"
#CAM_ID = "DigiCam"
#CAM_ID = "FlashCam"
#CAM_ID = "NectarCam"
CAM_ID = "LSTCam"

IMG_ID_LIST = [
    "run104_2081200_1",
    "run105_1244901_4",
]
#IMG_ID_LIST = []

TEL_FILTER_LIST = []
EVENT_FILTER_LIST = []

In [ ]:
if len(IMG_ID_LIST) > 0:
    
    # Use only the images defined in IMG_ID_LIST
    
    USE_FITS = False
    FITS_DIR = "~/data/grid_prod3b_north/fits/lst/gamma"
    SIMTEL_DIR = "~/data/grid_prod3b_north/simtel/gamma"

    PATHS = []
    for img_id in IMG_ID_LIST:
        run_id, event_id, tel_id = img_id.split("_")
        
        run_id = int(run_id[3:])
        event_id = int(event_id)
        tel_id = int(tel_id)
        
        if USE_FITS:
            PATHS.append("{}/gamma_20deg_0deg_run{}___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz_TEL{:03d}_EV{}.fits".format(FITS_DIR, run_id, tel_id, event_id))
        else:
            PATHS.append("{}/gamma_20deg_0deg_run{}___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz".format(SIMTEL_DIR, run_id))
            TEL_FILTER_LIST.append(tel_id)
            EVENT_FILTER_LIST.append(event_id)
            
    NUM_IMAGES = None
                
else:
    
    # Use the N first images in the following files
    
    #SIMTEL_FILE = "~/data/astri_mini_array_konrad/simtel/astri_v2/gamma/gamma_20deg_180deg_run2203___cta-prod3-sst-astri_desert-2150m-Paranal-sst-astri2.simtel.gz"
    #SIMTEL_FILE = "~/data/gct_mini_array_konrad/simtel/gct/gamma/gamma_20deg_180deg_run2203___cta-prod3-sst-gct_desert-2150m-Paranal-sst-gct.simtel.gz"
    #SIMTEL_FILE = "~/data/sst1m_mini_array_konrad/simtel/sst1m/gamma/gamma_20deg_180deg_run2203___cta-prod3-sst-dc_desert-2150m-Paranal-sst-dc.simtel.gz"
    SIMTEL_FILE = "~/data/grid_prod3b_north/simtel/gamma/gamma_20deg_0deg_run104___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz"
    PATHS = [SIMTEL_FILE]
    
    NUM_IMAGES = 30
    
print(PATHS)

## Define preselection cuts

In [ ]:
#rejection_criteria = lambda ref_image: not 200 < np.nansum(ref_image) < 250
#rejection_criteria = CTAMarsCriteria(cam_id=CAM_ID)
rejection_criteria = None

## Get images

In [ ]:
integrator_config_dict = get_mars_like_default_integrator_config(CAM_ID)

image_dict = {"run{}_{}_{}".format(int(image.meta['run_id']),
                                   int(image.meta['event_id']),
                                   int(image.meta['tel_id'])): image
              for image
              in image_generator(PATHS,
                                 max_num_images=NUM_IMAGES,
                                 cam_filter_list=[CAM_ID],
                                 tel_filter_list=TEL_FILTER_LIST,
                                 ev_filter_list=EVENT_FILTER_LIST,
                                 ctapipe_format=False,
                                 mc_rejection_criteria=rejection_criteria,
                                 **integrator_config_dict)}

In [ ]:
if len(IMG_ID_LIST) > 0:
    image_dict = {k: v for k, v in image_dict.items() if k in IMG_ID_LIST}

In [ ]:
@interact(image_key=image_dict.keys(), picture_thresh=10, boundary_thresh=5, pixels_clusters=["off", "scipy", "mars"])
def compute_hillas_and_display(image_key, picture_thresh=10, boundary_thresh=5, pixels_clusters="off"):
    image = image_dict[image_key]
    calibrated_image = image.input_image
    cam_id = image.meta['cam_id']
    
    tailcut = Tailcut()
    tailcut_cleaned_img = tailcut.clean_image(calibrated_image,
                                              high_threshold=picture_thresh,
                                              low_threshold=boundary_thresh,
                                              pixels_clusters_filtering=pixels_clusters,
                                              cam_id=cam_id)
    
    if np.nanmax(tailcut_cleaned_img) == 0:
        print("Empty image")
    else:
        calibrated_image_1d = geometry_converter.image_2d_to_1d(calibrated_image, cam_id)
        tailcut_cleaned_img_1d = geometry_converter.image_2d_to_1d(tailcut_cleaned_img, cam_id)
        geom1d = geometry_converter.get_geom1d(cam_id)
        
        tailcut_cleaned_img_mask = np.full(tailcut_cleaned_img_1d.shape, False)
        tailcut_cleaned_img_mask[tailcut_cleaned_img_1d > 0] = True

        hillas_params = get_hillas_parameters(geom1d, tailcut_cleaned_img_1d, implementation=4)

        disp = plot_ctapipe_image(calibrated_image_1d, geom1d, title='Tailcut cleaned', plot_axis=False)
        plot_hillas_parameters_on_axes(disp.axes, tailcut_cleaned_img_1d, geom1d)
        disp.highlight_pixels(tailcut_cleaned_img_mask, linewidth=4, color='white', alpha=0.9)
        plt.savefig('tailcut_cleaned_img.pdf')
        disp.show()
        
        disp = plot_ctapipe_image(tailcut_cleaned_img_1d, geom1d, title='Tailcut cleaned', plot_axis=False)
        plot_hillas_parameters_on_axes(disp.axes, tailcut_cleaned_img_1d, geom1d)
        disp.show()
        
        ### Ref ###
        
        reference_image_1d = geometry_converter.image_2d_to_1d(image.reference_image, cam_id)
        reference_image_mask = np.full(reference_image_1d.shape, False)
        reference_image_mask[reference_image_1d > 0] = True
        disp = plot_ctapipe_image(reference_image_1d, geom1d, title='Reference', plot_axis=False)
        plot_hillas_parameters_on_axes(disp.axes, reference_image_1d, geom1d)
        disp.highlight_pixels(reference_image_mask, linewidth=4, color='white', alpha=0.9)
        disp.show()

        print("cen_x:...", hillas_params.cen_x)
        print("cen_y:...", hillas_params.cen_y)

        print("length:..", hillas_params.length)
        print("width:...", hillas_params.width)

        print("size:....", hillas_params.size, "PE")
        print("NPE: ....", np.sum(tailcut_cleaned_img), "PE")

        print("psi:.....", hillas_params.psi)

        print("miss:....", hillas_params.miss)
        print("phi:.....", hillas_params.phi)
        print("r:.......", hillas_params.r)

        print("kurtosis:", hillas_params.kurtosis)
        print("skewness:", hillas_params.skewness)